This notebook demonstrates ingesting ENCODE bigWig into tileDB dense array with S3 backend.

# Why use tileDB?
With TileDB you gain the ability to quickly query array-structured data using rectangular slices, quickly update existing arrays with new or changed data, and easily optimize your physical data organization for maximizing compression and read performance.

# What is bigWig
https://genome.ucsc.edu/goldenPath/help/bigWig.html

# How to configure S3 backend
https://docs.tiledb.io/en/stable/tutorials/working-with-s3.html
Create an s3 bucket. In this example the bucket will be at s3://tile-db-test. In this example the bucket is in the region us-west-2.

In [1]:
s3_base_url = 's3://tile-db-test/'

# Open bigWig with pyBigWig
ENCODE bigWig files can be accessed in the ENCODE public s3 bucket. pyBigWig can open files directly from URLs.

In [2]:
import pyBigWig
bw = pyBigWig.open('https://encode-public.s3.amazonaws.com/2019/02/15/d5c222de-f74d-4ac7-b056-b3b5d159c773/ENCFF111MVN.bigWig')
bw_base_name = 'ENCFF111MVN'
chrom_info = bw.chroms()

# Write data to tileDB

In [ ]:
import tiledb
import numpy as np
# Configure the region
config = tiledb.Config()
config['vfs.s3.region'] = 'us-west-2'
# Set the address for the base url where the arrays will be written. There is going to be an array per chromosome.
tiledb_s3_baseaddress = s3_base_url + bw_base_name + '/'
# Start building the tiledb
ctx = tiledb.Ctx(config)

def write_array_to_tiledb(array, url, ctx, default_tile_size=9000, compressor='gzip', compression_level=-1):
    size = array.shape[0]
    tile_size = min(size, default_tile_size)
    tiledb_dim = tiledb.Dim(ctx, name='genome_coordinate', domain=(0, size - 1), tile=tile_size, dtype='uint32')
    tiledb_dom = tiledb.Domain(ctx, tiledb_dim)
    tiledb_attr = tiledb.Attr(ctx, name='signal_value', compressor=(compressor, compression_level), dtype='float32')
    tiledb_schema = tiledb.ArraySchema(ctx, domain=tiledb_dom, attrs=(tiledb_attr,),
                                       cell_order='row-major', tile_order='row-major')
    tiledb.DenseArray.create(url, tiledb_schema)
    with tiledb.DenseArray(ctx, url, mode='w') as s3array:
        array = array.astype(np.float32)
        s3array[:] = array

# iterate over chromosomes and create the tiledb array in the s3 bucket for each.
for chrom_name, chrom_size in chrom_info.items():
    signal_data = np.zeros(chrom_size, dtype=np.float32)
    signal_data[:] = bw.values(chrom_name, 0, chrom_size)
    chrom_s3_address = tiledb_s3_baseaddress + chrom_name
    write_array_to_tiledb(signal_data, chrom_s3_address, ctx)



# Query data